# OpenFOAM HRM Log Analysis

This program plots data from HRMFoam log files for analysis (i.e. net mass flux).
The data should already be extracted to HDF using ReadHRMLog
    
    @author Daniel Duke <daniel.duke@monash.edu>
    @copyright (c) 2020 LTRAC
    @license GPL-3.0+
    @version 0.0.1
    @date 02/11/2022
        __   ____________    ___    ______
       / /  /_  ____ __  \  /   |  / ____/
      / /    / /   / /_/ / / /| | / /
     / /___ / /   / _, _/ / ___ |/ /_________
    /_____//_/   /_/ |__\/_/  |_|\__________/

    Laboratory for Turbulence Research in Aerospace & Combustion (LTRAC)
    Monash University, Australia


In [1]:
# Load modules
import numpy as np
import glob, copy, gzip, natsort, tqdm, os
import h5py
#import matplotlib.pyplot as plt
#%matplotlib notebook